In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [57]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

In [58]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

test_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

In [59]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100)

In [100]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fc1 = nn.Linear(in_features=12*20*20, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        t = F.relu( self.conv1(t) )
        t = F.relu( self.conv2(t) )
        t = t.reshape(-1, 12*20*20)
        t = F.relu( self.fc1(t) )
        t = F.relu( self.fc2(t) )
        t = nn.Softmax( self.out(t) )
        return t

In [101]:
model = Network().to(device)
criterion = nn.CrossEntropyLoss
optimizer = torch.optim.Adam( model.parameters() )

In [108]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [111]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += criterion(output, target).data
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [112]:
for epoch in range(1, 5):
    train(epoch)
    test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.293362
Train Epoch: 1 [1000/60000 (2%)]	Loss: 0.373800
Train Epoch: 1 [2000/60000 (3%)]	Loss: 0.372548
Train Epoch: 1 [3000/60000 (5%)]	Loss: 0.389119
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.566627
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.473685
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.350877
Train Epoch: 1 [7000/60000 (12%)]	Loss: 0.385277
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.424888
Train Epoch: 1 [9000/60000 (15%)]	Loss: 0.367853
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.330080
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.441504
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.372166
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.252297
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.385471
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.384057
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.509475
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.362353
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.559916
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.551835
Train Epoch: 1 [200

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0039, Accuracy: 8614/10000 (86%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.232892
Train Epoch: 2 [1000/60000 (2%)]	Loss: 0.345506
Train Epoch: 2 [2000/60000 (3%)]	Loss: 0.297072
Train Epoch: 2 [3000/60000 (5%)]	Loss: 0.333368
Train Epoch: 2 [4000/60000 (7%)]	Loss: 0.454411
Train Epoch: 2 [5000/60000 (8%)]	Loss: 0.380035
Train Epoch: 2 [6000/60000 (10%)]	Loss: 0.338062
Train Epoch: 2 [7000/60000 (12%)]	Loss: 0.333245
Train Epoch: 2 [8000/60000 (13%)]	Loss: 0.382075
Train Epoch: 2 [9000/60000 (15%)]	Loss: 0.329874
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.289463
Train Epoch: 2 [11000/60000 (18%)]	Loss: 0.331200
Train Epoch: 2 [12000/60000 (20%)]	Loss: 0.297945
Train Epoch: 2 [13000/60000 (22%)]	Loss: 0.214009
Train Epoch: 2 [14000/60000 (23%)]	Loss: 0.321892
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.319846
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.411200
Train Epoch: 2 [17000/60000 (28%)]	Loss: 0.335378
Train Epoch: 2 [18000/60000 (30%)]	Loss: 0.483607
Train Ep

Train Epoch: 4 [42000/60000 (70%)]	Loss: 0.357043
Train Epoch: 4 [43000/60000 (72%)]	Loss: 0.238609
Train Epoch: 4 [44000/60000 (73%)]	Loss: 0.171482
Train Epoch: 4 [45000/60000 (75%)]	Loss: 0.272259
Train Epoch: 4 [46000/60000 (77%)]	Loss: 0.211548
Train Epoch: 4 [47000/60000 (78%)]	Loss: 0.240014
Train Epoch: 4 [48000/60000 (80%)]	Loss: 0.324622
Train Epoch: 4 [49000/60000 (82%)]	Loss: 0.311308
Train Epoch: 4 [50000/60000 (83%)]	Loss: 0.319958
Train Epoch: 4 [51000/60000 (85%)]	Loss: 0.187180
Train Epoch: 4 [52000/60000 (87%)]	Loss: 0.220615
Train Epoch: 4 [53000/60000 (88%)]	Loss: 0.198170
Train Epoch: 4 [54000/60000 (90%)]	Loss: 0.251251
Train Epoch: 4 [55000/60000 (92%)]	Loss: 0.274135
Train Epoch: 4 [56000/60000 (93%)]	Loss: 0.283182
Train Epoch: 4 [57000/60000 (95%)]	Loss: 0.345591
Train Epoch: 4 [58000/60000 (97%)]	Loss: 0.285626
Train Epoch: 4 [59000/60000 (98%)]	Loss: 0.204527

Test set: Average loss: 0.0034, Accuracy: 8759/10000 (87%)

